In [1]:
import pandas as pd
import numpy as np
import time
import cantera as ct
from matplotlib import pyplot as plt
import csv
import math
import os
import sys
import re
import itertools
import logging
from collections import defaultdict
import git
import json

from rmgpy.molecule import Molecule
from rmgpy.data.base import Database

In [9]:
#######################################################################
# Input Parameters for combustor
#######################################################################

# filepath for writing files
git_repo = "../ammonia/"
cti_file = git_repo + "base/cantera/chem_annotated.cti"

# Reactor settings arrays for run
temp = [400, 500, 600,700,800,900,1000,1100,1200,1300,1400,1500]
pressure = 0.08 # 1 bar
volume_flows = 1.6666666666667E-7 #10 ml/min = 1.6666666666667E-7 m^3/s

X_o2 = 0.02 #O2 partial pressure(atm)
X_nh3 = 0.01
X_h2o = 0.05
# reaction time
reactime = 1e5

# sensitivity settings
sensitivity = False
sensatol = 1e-6
sensrtol = 1e-6

# Specify idealGasReactor and isothermal
reactor_type=1
energy="off"

# set sensitivity string for file path name
if sensitivity:
    sensitivity_str = "on"
else: 
    sensitivity_str = "off"
mw_nh3 = 17.0306e-3  # [kg/mol]
mw_o2 = 31.999e-3  # [kg/mol]
mw_h2o = 18.015e-3  # [kg/mol]
   
o2_ratio = X_nh3 / X_o2
# O2/NH3/He: typical is
concentrations_rmg = {"O2(2)": X_o2, "NH3(6)": X_nh3, "H2O(3)": X_h2o}

 
# initialize cantera gas and surface
gas = ct.Solution(cti_file, "gas")
surf = ct.Interface(cti_file, "surface1", [gas])

# initialize T and P
gas.TPX = temp, pressure, concentrations_rmg
surf.TP = temp, pressure # change this to surf_temp when we want a different starting temperature for the surface

# if a mistake is made with the input, 
# cantera will normalize the mole fractions. 
# make sure that we are reporting/using 
# the normalized values
X_o2 = float(gas["O2(2)"].X)
X_nh3 = float(gas["NH3(6)"].X)
X_h2o = float(gas["H2O(3)"].X)


# create gas inlet
inlet = ct.Reservoir(gas)

# create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume
rvol = 1.4711 #m3

# Catalyst Surface Area
site_density = (surf.site_density * 1000)  # [mol/m^2]cantera uses kmol/m^2, convert to mol/m^2
cat_area = 6.5461  #1.387*1.5023*3.1416=6.5461[m^3] 

# reactor initialization
if reactor_type == 0:
    r = ct.Reactor(gas, energy=energy)
    reactor_type_str = "Reactor"
elif reactor_type == 1:
    r = ct.IdealGasReactor(gas, energy=energy)
    reactor_type_str = "IdealGasReactor"
elif reactor_type == 2:
    r = ct.ConstPressureReactor(gas, energy=energy)
    reactor_type_str = "ConstPressureReactor"
elif reactor_type == 3:
    r = ct.IdealGasConstPressureReactor(gas, energy=energy)
    reactor_type_str = "IdealGasConstPressureReactor"

# calculate the available catalyst area in a differential reactor
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol
surf.coverages = "X(1):1.0"

# flow controllers
one_atm = ct.one_atm
FC_temp = 293.15
volume_flow = settings[array_i][3]  # [m^3/s]
molar_flow = volume_flow * one_atm / (8.3145 * FC_temp)  # [mol/s]
mass_flow = molar_flow * (X_nh3 * mw_nh3 + X_o2 * mw_o2 + X_h2o * mw_h2o)  # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(r, exhaust, master=mfc, K=0.01)

# initialize reactor network
sim = ct.ReactorNet([r])

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-15

#################################################
# Run single reactor
#################################################

# round numbers so they're easier to read
# temp_str = '%s' % '%.3g' % tempn
cat_area_str = "%s" % "%.3g" % cat_area

results_path = (
    os.path.dirname(os.path.abspath(__file__))
   + f"/results"
)
results_path_csp = (
    os.path.dirname(os.path.abspath(__file__))
   + f"/results/csp"
)

# create species folder for species pictures if it does not already exist
try:
    os.makedirs(species_path, exist_ok=True)
    save_pictures(git_path=rmg_model_path, species_path=species_path)
except OSError as error:
    print(error)

try:
    os.makedirs(results_path, exist_ok=True)
except OSError as error:
    print(error)

try:
    os.makedirs(results_path_csp, exist_ok=True)
    print(results_path_csp)
except OSError as error:
    print(error)
    print("no results for CSP saved")

try:
    os.makedirs(flux_path, exist_ok=True)
except OSError as error:
    print(error)

gas_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.species_names]

# surface ROP reports gas and surface ROP. these values might be redundant, not sure.

gas_surf_ROP_str = [i + " surface ROP [kmol/m^2 s]" for i in gas.species_names]
surf_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.species_names]

gasrxn_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.reaction_equations()]
surfrxn_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.reaction_equations()]

output_filename = (
    results_path
    + f"/Spinning_basket_area_{cat_area_str}_energy_{energy}"
    + f"_temp_{temp}_O2_{x_O2_str}_NH3_{x_NH3_str}.csv"
)
output_filename_csp = (
    results_path_csp
    + f"/CSP_Spinning_basket_area_{cat_area_str}_energy_{energy}"
    + f"_temp_{temp}_O2_{x_O2_str}_NH3_{x_NH3_str}.csv"
)
outfile = open(output_filename, "w")
outfile_csp = open(output_filename_csp, "w")
writer = csv.writer(outfile)
writer_csp = csv.writer(outfile_csp)

# Sensitivity atol, rtol, and strings for gas and surface reactions if selected
# slows down script by a lot
if sensitivity:
    sim.rtol_sensitivity = sensrtol
    sim.atol_sensitivity = sensatol
    sens_species = ["NH3(6)", "O2(2)", "N2(4)", "NO(5)", "N2O(7)"]  #change THIS to your species, can add "," and other species

    # turn on sensitive reactions/species
    for i in range(gas.n_reactions):
        r.add_sensitivity_reaction(i)

    for i in range(surf.n_reactions):
        rsurf.add_sensitivity_reaction(i)

    # for i in range(gas.n_species):
    #     r.add_sensitivity_species_enthalpy(i)

    # for i in range(surf.n_species):
    #     rsurf.add_sensitivity_species_enthalpy(i)

    for j in sens_species:
        gasrxn_sens_str = [
            j + " sensitivity to " + i for i in gas.reaction_equations()
        ]
        surfrxn_sens_str = [
            j + " sensitivity to " + i for i in surf.reaction_equations()
        ]
        # gastherm_sens_str = [j + " thermo sensitivity to " + i for i in gas.species_names]
        # surftherm_sens_str = [j + " thermo sensitivity to " + i for i in surf.species_names]
        sens_list = gasrxn_sens_str + surfrxn_sens_str  # + gastherm_sens_str

    writer.writerow(
        [
            "T (K)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial",
            "X_o2 initial",
            "X_h2o initial",
            "(NH3/O2)",
            "T (K) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
        + gasrxn_ROP_str
        + surfrxn_ROP_str
        + sens_list
    )

else:
    writer.writerow(
        [
            "T (K)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial",
            "X_o2 initial",
            "X_h2o initial",
            "(NH3/O2)",
            "T (K) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
        + gasrxn_ROP_str
        + surfrxn_ROP_str
    )

writer_csp.writerow(
    ["iter", "t", "dt", "Density[kg/m3]", "Pressure[Pascal]", "Temperature[K]",]
    + gas.species_names
    + surf.species_names
)

t = 0.0
dt = 0.1
iter_ct = 0
# run the simulation
first_run = True

while t < reactime:
    # save flux diagrams at beginning of run
    if first_run == True:
        save_flux_diagrams(gas, suffix=flux_path, timepoint="beginning", species_path=species_path)
        save_flux_diagrams(surf, suffix=flux_path, timepoint="beginning", species_path=species_path)

        first_run = False
    t += dt
    sim.advance(t)
#         if t % 10 < 0.01:

    if sensitivity:
        # get sensitivity for sensitive species i (e.g. methanol) in reaction j
        for i in sens_species:
            g_nrxn = gas.n_reactions
            s_nrxn = surf.n_reactions
            # g_nspec = gas.n_species
            # s_nspec = surf.n_species

            gas_sensitivities = [sim.sensitivity(i, j) for j in range(g_nrxn)]
            surf_sensitivities = [
                sim.sensitivity(i, j) for j in range(g_nrxn, g_nrxn + s_nrxn)
            ]
            # gas_therm_sensitivities = [sim.sensitivity(i,j)
            # for j in range(g_nrxn+s_nrxn,g_nrxn+s_nrxn+g_nspec)]
            # surf_therm_sensitivities = [sim.sensitivity(i,j)
            # for j in range(g_nrxn+s_nrxn+g_nspec,g_nrxn+s_nrxn+g_nspec+s_nspec)]

            sensitivities_all = (
                gas_sensitivities
                + surf_sensitivities
                # + gas_therm_sensitivities
            )

        writer.writerow(
            [
                temp,
                pressure,
                volume_flow,
                X_nh3,
                X_o2,
                X_h2o,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
            + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
            + sensitivities_all
        )

    else:
        writer.writerow(
            [
                temp,
                pressure,
                volume_flow,
                X_nh3,
                X_o2,
                X_h2o,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
            + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
        )

    writer_csp.writerow(
        [
            iter_ct,
            sim.time,
            dt,
            gas.density,
            gas.P,
            gas.T,
        ]
        + list(gas.X)
        + list(surf.X)
    )

    iter_ct += 1

outfile.close()
outfile_csp.close()


TypeError: must be real number, not list